#### Business Analytics FHDW 2025
## Übung

1. Untersuchen Sie den Gebrauchtwagen-Datensatz auf redundante Variablen. Wenn Sie Variablen entfernen, versuchen Sie das sowohl aus dem Domänenkontext, als auch abstrakt anhand der Zahlen zu begründen.

2. Führen Sie mit dem reduzierten Datensatz eine Regression zur Vorhersage des Preises durch. Vergleichen Sie die Performance Ihres reduzierten Modells mit der des Modells mit dem vollen Umfang.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pylab as plt
import seaborn as sns
import numpy as np
import math

def regressionSummary(y_true, y_predicted):
    y_true = np.asarray(y_true)
    y_predicted = np.asarray(y_predicted)
    y_residuals = y_true - y_predicted
    metrics = [
        ('Summe Abweichungen', sum(y_residuals)),
        ('Summe absolute Abweichungen', sum(abs(y_residuals))),
        ('Mittlerer Fehler', sum(y_residuals) / len(y_residuals)),
        ('Mittlerer absoluter Fehler', sum(abs(y_residuals)) / len(y_residuals)),
        ('Wurzel des durchschnittlichen Fehlerquadrats', math.sqrt(mean_squared_error(y_true, y_predicted)))
    ]
    if all(yt != 0 for yt in y_true):
        metrics.extend([
            ('Mittlerer prozentualer Fehler', 100 * sum(y_residuals / y_true) / len(y_residuals)),
            ('Mittlerer absoluter prozentualer Fehler', 100 * sum(abs(y_residuals / y_true) / len(y_residuals))),
        ])
    maxlength = max(len(m[0]) for m in metrics)
    fmt1 = f'{{:>{maxlength}}} : {{:.4f}}'
    print('\nRegressionskennzahlen\n')
    for metric, value in metrics:
        print(fmt1.format(metric, value))

car_df = pd.read_csv('Daten/ToyotaCorolla.csv')
car_df = car_df.iloc[0:1000]

predictors = ['age_08_04', 
              'km', 
              'fuel_type', 
              'hp', 
              'met_color', 
              'automatic', 
              'cc', 
              'doors', 
              'quarterly_tax', 
              'weight']
outcome = 'price'        

X = pd.get_dummies(car_df[predictors], drop_first=True)
y = car_df[outcome]

X

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(11, 7)
sns.heatmap(X.corr(), annot=True, fmt=".1f", cmap="RdBu", center=0, ax=ax)

Die beiden Kraftstoffarten korrelieren sehr stark. Gleichzeitig sind beide stark abhängig von der Steuer und dem Gewicht. Nehmen wir die Kraftstoffarten also raus, da ihre Informationen hinreichend in Steuer und Gewicht enthalten sind.

In [ ]:
#X_reduced = X[X.columns.difference(["fuel_type_Diesel", "fuel_type_Petrol", "quarterly_tax"])]
print(X.columns.difference(["fuel_type_Diesel", "fuel_type_Petrol"]))
X_reduced = X[X.columns.difference(["fuel_type_Diesel", "fuel_type_Petrol"])]
X_reduced

Qualität der Anpassung an Trainingsdaten:

In [ ]:
train_X_reduced, valid_X_reduced, train_y, valid_y = train_test_split(X_reduced, y, test_size=0.4, random_state=1)
car_lr_reduced = LinearRegression()
car_lr_reduced.fit(train_X_reduced, train_y)

print(pd.DataFrame({'Prediktor': X_reduced.columns, 'Koeffizient': car_lr_reduced.coef_}))
regressionSummary(train_y, car_lr_reduced.predict(train_X_reduced))

Qualität der Vorhersage auf Basis des reduzierten Modells:

In [ ]:
car_lr_reduced_prediction = car_lr_reduced.predict(valid_X_reduced)
#print(car_lr_reduced_prediction)
residuals = valid_y - car_lr_reduced_prediction
result = pd.DataFrame({'Vorhersage': car_lr_reduced_prediction, 'Tatsächlich': valid_y, 'Residuum': residuals})
print(result.head(20))
regressionSummary(valid_y, car_lr_reduced_prediction)